In [ ]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import learning_curve

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('/kaggle/input/cibil-data/cibil_data.csv')

In [ ]:
X = df.drop(['cibil_score', 'score_category'], axis=1)
y = df['score_category']  # For classification
# OR
y = df['cibil_score']  # For regression

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Load dataset
df = pd.read_csv('/kaggle/input/cibil-data/cibil_data.csv')

# Separate features and target
X = df.drop(['cibil_score', 'score_category'], axis=1)
y = df['score_category']  # This has strings: 'Poor', 'Fair', 'Good', 'Excellent'

# CRITICAL STEP: Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Train XGBoost (will work now!)
model = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Convert predictions back to original labels
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

print(f"Predictions: {y_pred_labels[:10]}")
# Output: ['Good' 'Excellent' 'Poor' ...]


In [ ]:
# Check - model exists
if 'model' not in dir() or 'label_encoder' not in dir():
    print(" Error: Please run the training cell first!")
else:
    print(" Model found in memory!\n")
    
    # input layer
    print("📋 Enter your information:\n")
    
    try:
        num_timely_payments = int(input("1️. Timely Payments (0-120): "))
        num_defaults = int(input("2️. Defaults (0-10): "))
        credit_utilization = float(input("3️. Credit Utilization % (0-100): "))
        credit_history_years = int(input("4️. Credit History Years (0-35): "))
        num_inquiries = int(input("5️. Credit Inquiries (0-20): "))
        monthly_income = int(input("6️. Monthly Income ₹ (15000-500000): "))
        
        
        # Calculate derived values
        payment_consistency = (num_timely_payments / (num_timely_payments + num_defaults + 1)) * 100
        
        # Build features in EXACT ORDER as training data
        features = pd.DataFrame([{
            'num_timely_payments': num_timely_payments,
            'num_delayed_payments': max(0, int(num_defaults * 0.5)),
            'num_defaults': num_defaults,
            'payment_history_months': credit_history_years * 12,
            'credit_utilization_ratio': credit_utilization,
            'credit_limit_total': int(monthly_income * 3),
            'outstanding_balance': int(monthly_income * 3 * credit_utilization / 100),
            'num_credit_cards': min(max(1, num_timely_payments // 20), 8),
            'credit_history_years': credit_history_years,
            'age_oldest_account_years': int(credit_history_years * 0.8),
            'avg_account_age_years': int(credit_history_years * 0.6),
            'num_bank_accounts': 3,
            'num_loans': 2,
            'has_mortgage': 1 if monthly_income > 50000 else 0,
            'has_auto_loan': 1 if monthly_income > 40000 else 0,
            'loan_types_count': 3,
            'num_credit_inquiries_6m': num_inquiries,
            'num_new_accounts_12m': min(num_inquiries, 3),
            'months_since_last_inquiry': 6,
            'age': min(max(25, credit_history_years + 22), 65),
            'monthly_income': monthly_income,
            'employment_length_years': min(credit_history_years, 20),
            'total_debt': int(monthly_income * 3 * credit_utilization / 100),
            'debt_to_income_ratio': credit_utilization * 3,
            'payment_consistency_score': payment_consistency,
            'income_stability_score': min(50, credit_history_years * 5)
        }])
        
        # CRITICAL: Reorder columns to match training data
        training_columns = X_train.columns.tolist()
        features = features[training_columns]
        
        # Make prediction
        print("🔮 Predicting CIBIL score...\n")
        prediction_encoded = model.predict(features)
        prediction_proba = model.predict_proba(features)
        
        # Get category
        predicted_category = label_encoder.inverse_transform(prediction_encoded)[0]
        
        # Map to score
        score_map = {
            'Poor': np.random.randint(350, 550),
            'Fair': np.random.randint(550, 650),
            'Good': np.random.randint(650, 750),
            'Excellent': np.random.randint(750, 850)
        }
        predicted_score = score_map[predicted_category]
        
        # Display results
        
        emoji_map = {'Poor': '🔴', 'Fair': '🟡', 'Good': '🟢', 'Excellent': '⭐'}
        print(f"\n{emoji_map[predicted_category]} Predicted CIBIL Score: {predicted_score}")
        print(f"📊 Category: {predicted_category}")
        
        print("\n📈 Score Guide:")
        print("   300-549: Poor | 550-649: Fair | 650-749: Good | 750-900: Excellent")
        
        print("\n🔍 Model Confidence:")
        for i, cls in enumerate(label_encoder.classes_):
            conf = prediction_proba[0][i] * 100
            bar = '█' * int(conf/2) + '░' * (50 - int(conf/2))
            print(f"   {cls:<12}: {bar} {conf:.1f}%")
        
        print("\n" + "="*70)
        
    except ValueError as e:
        print(f"\n Invalid input: {e}")
